In [1]:
from fitbit.auth import get_tokens, get_fitbit_client
from datetime import datetime, timedelta
# %pip install pandas
import pandas as pd
import json

access_token, refresh_token = get_tokens()
fitbit_client = get_fitbit_client(access_token, refresh_token)

In [2]:
today = datetime.today().strftime('%Y-%m-%d')
last_week = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%d')
last_month = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')
last_year = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')

# Annual Steps

In [ ]:
import pandas as pd

df = pd.DataFrame(fitbit_client.get_activity_time_series_range(
    resource='steps', 
    start_date=today,
    end_date=last_year
)['activities-steps'])

In [ ]:
df['value'] = df['value'].astype(int)
df[df['value'] > 0]


# Personal Best Stats

In [ ]:
activity_stats = fitbit_client.activity_stats()
steps_best_day = activity_stats['best']['total']['steps']['value']
steps_best_day_date = activity_stats['best']['total']['steps']['date']
distance_lifetime = activity_stats['best']['total']['distance']['value']

print(f"Highest Step Count: {steps_best_day}")
print(f"Date Achieved: {steps_best_day_date}")
print(f"Total distance: {round(distance_lifetime, 2)} KM")

# Personal Lifetime Stats

In [ ]:
lifetime_distance = activity_stats['lifetime']['total']['distance']
lifetime_steps = activity_stats['lifetime']['total']['steps']

print(f"Total distance: {round(lifetime_distance, 2)} KM")
print(f"Total steps: {lifetime_steps}")

# User Profile

In [25]:
user_profile = fitbit_client.get_user_profile()['user']

In [ ]:
badges_df = pd.DataFrame(user_profile['topBadges'])
badges_df

In [ ]:
# not user_profile minus topBadges
user_profile_minus_badges = {k: v for k, v in user_profile.items() if k != 'topBadges'}
user_profile_df = pd.DataFrame(user_profile_minus_badges)
user_profile_df

# Sleep

In [ ]:
sleep_data = fitbit_client.get_sleep(date=datetime.today())

# Sleep Data
sleep_df = pd.DataFrame(sleep_data['sleep'])

# Sleep Stages Summary
sleep_stages_summary = sleep_data['summary']['stages']
deep_sleep_duration = sleep_stages_summary['deep']
light_sleep_duration = sleep_stages_summary['light']
rem_sleep_duration = sleep_stages_summary['rem']
wake_sleep_duration = sleep_stages_summary['wake']

# append to sleep_df
sleep_df['deep_sleep_duration'] = deep_sleep_duration
sleep_df['light_sleep_duration'] = light_sleep_duration
sleep_df['rem_sleep_duration'] = rem_sleep_duration
sleep_df['wake_sleep_duration'] = wake_sleep_duration

sleep_minute_df = pd.DataFrame(sleep_df['minuteData'][0])

# try to remove minuteData from sleep_df    
try:    
    sleep_df = sleep_df.drop(columns=['minuteData'])
except KeyError:
    print("KeyError: minuteData not found")
    
sleep_df.T

In [ ]:
# group by value and count
sleep_minute_df

# Recent Activities

In [68]:
"""
# Resource Options
All Activity	Tracker Only Activity
----------------------------------
activityCalories	tracker/activityCalories
calories	tracker/calories
caloriesBMR	N/A
distance	tracker/distance
elevation	tracker/elevation
floors	tracker/floors
minutesSedentary	tracker/minutesSedentary
minutesLightlyActive	tracker/minutesLightlyActive
minutesFairlyActive	tracker/minutesFairlyActive
minutesVeryActive	tracker/minutesVeryActive
steps	tracker/steps
swimming-strokes	N/A
"""

period = '7d' # 1d | 7d | 30d | 1w | 1m | 3m | 6m | 1y
activity_calories = fitbit_client.get_activity_time_series(resource='activityCalories', date=today, period=period)
calories = fitbit_client.get_activity_time_series(resource='calories', date=today, period=period)
distance = fitbit_client.get_activity_time_series(resource='distance', date=today, period=period)
elevation = fitbit_client.get_activity_time_series(resource='elevation', date=today, period=period)
floors = fitbit_client.get_activity_time_series(resource='floors', date=today, period=period)
minutes_sedentary = fitbit_client.get_activity_time_series(resource='minutesSedentary', date=today, period=period)
minutes_lightly_active = fitbit_client.get_activity_time_series(resource='minutesLightlyActive', date=today, period=period)
minutes_fairly_active = fitbit_client.get_activity_time_series(resource='minutesFairlyActive', date=today, period=period)
minutes_very_active = fitbit_client.get_activity_time_series(resource='minutesVeryActive', date=today, period=period)
steps = fitbit_client.get_activity_time_series(resource='steps', date=today, period=period)
swimming_strokes = fitbit_client.get_activity_time_series(resource='swimming-strokes', date=today, period=period)


In [70]:
dfs = [activity_calories, calories, distance, elevation, floors, minutes_sedentary, minutes_lightly_active, minutes_fairly_active, minutes_very_active, steps, swimming_strokes]

combined_df = []
for row in dfs:
    name = list(row.keys())[0]
#     new_row = {
#         'measure': name,
#         'value': row[name][0]['value'],
#         'date': row[name][0]['dateTime']
#     }
#     combined_df.append(new_row)
    
# combined_df = pd.DataFrame(combined_df)
# combined_df

,measure,value,date
0,activities-activityCalories,1289,2024-10-23
1,activities-calories,2945,2024-10-23
2,activities-distance,2.969357701,2024-10-23
3,activities-elevation,50,2024-10-23
4,activities-floors,5,2024-10-23
5,activities-minutesSedentary,1189,2024-10-23
6,activities-minutesLightlyActive,187,2024-10-23
7,activities-minutesFairlyActive,42,2024-10-23
8,activities-minutesVeryActive,22,2024-10-23
9,activities-steps,6216,2024-10-23
